# Battle of the Neighborhoods (Week 2)
#### Part of the IBM Data Science Certification: Applied Data Science Capstone, Final Project
    
## Abstract
    
In this work, we will be examining geographical locations from 3 different neighboring areas within northern Virginia. Each area will be divided into equally sized bounding boxes and the types of venues (shops, restaurants, ammenities) will be exmained to find the best fit based on a predetermined set of preferences. This is exercising a content-based recommender algorithm to determine the top `N` (in this case `N = 3`) sections of any of the neighborhoods. 


## Table of Contents

1. <a href="#introduction">Introduction (Background)</a>
1. <a href="#data_description">Data Description</a>
1. <a href="#methodology">Methodology</a>
1. <a href="#results">Results</a>
1. <a href="#discussion">Discussion</a>
1. <a href="#conclusion">Conclusion</a>

In [175]:
# The code was removed by Watson Studio for sharing.

In [176]:
%%capture
# Get stuff installed
!pip install geocoder
!pip install foursquare
!pip install folium
!pip install wordcloud

import pandas as pd
import numpy as np
from sklearn import preprocessing

# Geo-data
import geocoder
import foursquare
import folium # mapping

# Viz stuff
# Matplotlib and associated plotting modules
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
%matplotlib inline

# import package and its set of stopwords
from wordcloud import WordCloud, STOPWORDS
stopwords = set(STOPWORDS)


# Watson Studio stuff
from project_lib import Project

# Others
import re
import html
import math
from IPython.core.display import display, HTML

In [177]:
KM_PER_DEGREE = 111.32
R_EARTH = 6378.1

EXTRA_CATS = '4bf58dd8d48988d172941735,52f2ab2ebcbc57f1066b8b1d,5745c2e4498e11e7bccabdbd,4d954b0ea243a5684a65b473,4bf58dd8d48988d1fd931735,4bf58dd8d48988d116941735' # For some reason the foursquare api does not return post office or dry cleaners, drug stores, or convenience stores
CHANTILLY = 'chantilly'
TYSONS = 'tysons'
ARLINGTON = 'arlington'
ALEXANDRIA = 'alexandria'
PLACES = pd.DataFrame({
    'id': [CHANTILLY, TYSONS, ARLINGTON, ALEXANDRIA],
    'place': ['Chantilly, VA','Tysons Corner, VA', 'Arlington, VA', 'Alexandria, VA'], 
    'radius-m': [2780, 1660, 3500, 2000]
    })

PLACES = pd.concat([PLACES, pd.DataFrame([geocoder.google(place, key=api_key).latlng for place in PLACES['place']], columns=['lat', 'lng'])], axis=1)
CENTER = PLACES[['lat', 'lng']].mean()
BOUNDS = [list(PLACES[['lat','lng']].min().values), list(PLACES[['lat', 'lng']].max().values)]

FSQ_CLIENT = foursquare.Foursquare(client_id=fsq['id'], client_secret=fsq['sec'], version=fsq['ver'])
PROJECT = Project(None, proj['id'], proj['token'])

SURVEY = pd.read_csv(PROJECT.get_file('Survey.csv'))

In [178]:
# Functions

def newLatLng(lat, lng, distance, bearing):
    '''
    Generate new lat, lng based on input lat and lng, distance and bearing.
    
    lat, lng: current lat, lng in degrees
    d: distance in meters from current lat, lng
    bearing: 0 is north and west is -90 (270) and so on
    
    returns: (lat, lng)
    
    '''
    d = distance/1000
    lat1 = math.radians(lat)
    lng1 = math.radians(lng)
    
    brng = math.radians(bearing)
    lat2 = math.asin( math.sin(lat1)*math.cos(d/R_EARTH) + math.cos(lat1)*math.sin(d/R_EARTH)*math.cos(brng))
    lng2 = lng1 + math.atan2(math.sin(brng)*math.sin(d/R_EARTH)*math.cos(lat1), math.cos(d/R_EARTH)-math.sin(lat1)*math.sin(lat2))

    lat2 = math.degrees(lat2)
    lng2 = math.degrees(lng2)
    
    return (lat2, lng2)

def generateGrids(lat, lng, radius, grid_size, label):
    '''
    Generate a series of bounding boxes (Northing & Easting) with grid IDs (label_<N>) where N is a
    one-up counter from 0.
    
    lat, lng: location in degrees
    radius: in meters of entire region
    grid_size: in meters of a grid location (square, i.e. 500 would be a 500x500 square)
    label: some text, will result in IDs of label_<N>
    
    returns dataFrame of labels, Northing point and easting point
    '''
    
    # number of grids in one direction
    _grids = math.ceil(2*radius/grid_size)
    
    d = math.sqrt(grid_size*grid_size)
    input = {
        'grid': [],
        'boundN': [],
        'boundE': []
    }
    
    # we're just assuming a big square
    id = 0
    latTop, lngTop = newLatLng(lat, lng, radius, -45)
    curLat = latTop
    curLng = lngTop
    gLatE, gLngE = (latTop,0)
    for x in range(_grids):
        curLat = gLatE
        curLng = lngTop
        for y in range(_grids):
            id = y + _grids*x
            gLatE, gLngE = newLatLng(curLat, curLng, d, 135)
            input['grid'].append(f'{label}_{id}')
            input['boundN'].append((curLat, curLng))
            input['boundE'].append((gLatE, gLngE))
            curLng = gLngE
            
    return pd.DataFrame(input)
    

def getVenues(lat, lng, radius, **kwargs):
    params = {
        'll': f'{lat}, {lng}',
        'radius': radius
    }
        
    for key, value in kwargs.items():
        params[key] = value
        
    
    
    venues = []
    mvenues = 999999 if 'limit' not in params.keys() else limit
    offset = 0
    while offset < mvenues: 
        results = FSQ_CLIENT.venues.explore(params=params)
        mvenues = results['totalResults']
        vs = results['groups'][0]['items']
        offset += len(vs)
        params['offset'] = offset
        [venues.append(v) for v in vs if v['venue']['categories']]
        
    return venues
        
        
def venues2DataFrame(venues):
    vs = {
        'id': [],
        'venue': [],
        'category': [],
        'lat': [],
        'lng': []
    }
    for _v in venues:
        v = _v['venue']
        vs['id'].append(v['id'])
        vs['venue'].append(v['name'])
        vs['category'].append(v['categories'][0]['name'])
        vs['lat'].append(v['location']['lat'])
        vs['lng'].append(v['location']['lng'])
        
    return pd.DataFrame(vs)


def getAreaVenus(id, **kwargs):
    place = PLACES[PLACES['id'] == id].iloc[0,:]
    return venues2DataFrame(getVenues(place.loc['lat'], place.loc['lng'], place.loc['radius-m'], **kwargs))
    
    
def checkBounds(n, e, lat, lng):
    if (lat <= n[0]) and (lat > e[0]) and (lng >= n[1]) and (lng < e[1]):
        return True
    return False

<a id="introduction"></a>

## Introduction (Background)

ACME, Inc. is a growing company currently located in Chantilly, Virginia. Due to their growth, they need to relocate to a larger office and have decided to relocate closer to Washington, D.C. in response to their employees' preference. The owners of ACME, Inc. have determined that *Tysons Corner*, *Arlington*, or *Alexandria* would be all be viable locations for their new office. However, determining a specific location has become an issue. 

The following map shows the current location of ACME, Inc. (in red), and the other (blue) locations are the potential new areas.


In [179]:
map = folium.Map(location=CENTER, zoom_control=False, title='Test')
map.fit_bounds(BOUNDS)
for _, row in PLACES[['lat', 'lng', 'radius-m']].iterrows():
    folium.vector_layers.Circle(location=(row.loc['lat'], row.loc['lng']), radius=row.loc['radius-m'], fill=True, color='red' if _ == 0 else 'blue').add_to(map)


display(HTML('<h3 style="text-align: center">Potential Areas for ACME, Inc. Relocation</h3>'))    
map


In order to improve their employees' work experience, they issued an employee-wide survey to determine the types of venues, shops, restaurants or other ammenities that need to be near the new location (within approximately 500 meters or about 5 minutes walk). The following shows the results of their survey on a 10 point scale (1 = not interested, 10 = must have). 

In [180]:
SURVEY

,Type,Rating
0,Metro Station,9.0
1,Gym,6.5
2,Coffee Shop,7.0
3,Post Office,2.0
4,Cleaners,3.5
5,Sandwich Place,7.0
6,Convenience Store/Drug Store,2.5
7,Bar,8.5


<a id="data_description"></a>

## Data Description

Along with the employee survey data, we will need venues for the various areas. For this effort, we will retrieve venu data for the various grids [(see below)](#methodology) from the [Foursquare data set](https://foursquare.com/), using the [foursquare python library](https://pypi.org/project/foursquare/). The venu data will be matched to each grid ID (or `gid`) by the venues location and category (venues without categories will be removed). 

The following shows the first 10 venues and their categories from the Foursquare data set. Notice, that the category column will need some cleaning as it contains more specific categories than just `Restaurant`, e.g. `Sushi Restaurant`. Furthermore, there are venues denoted as `Pizza Place` which should be considered a `Restaurant`. 

In [181]:
tysons = pd.read_csv(PROJECT.get_file('tysons_10_venues.csv'))
tysons

,id,name,lat,lng,category
0,51891fea498ee05ee808e258,REI,38.918350,-77.228827,Sporting Goods Shop
1,4cd1b5b606b546881d3ce294,Super Chicken,38.920575,-77.235075,South American Restaurant
2,568441df498eed21c59b25de,Roll Play,38.916136,-77.227337,Vietnamese Restaurant
3,4a63e6acf964a520fbc51fe3,"Sakura Japanese Steak, Seafood House & Sushi Bar",38.921471,-77.235775,Sushi Restaurant
4,4b8323eff964a520f1f930e3,Fleming's Prime Steakhouse & Wine Bar,38.920557,-77.227068,Steakhouse
5,547bf26b498ea3fb9947b0cb,Esthetic Institute,38.914340,-77.234160,School
6,4f626ea7e4b0ea77cba053b8,CAVA,38.917194,-77.223629,Mediterranean Restaurant
7,5dd831fed892900007dfa839,Shotted Specialty Coffee,38.917435,-77.223452,Coffee Shop
8,59a5873fd3cce87c7c6cab7a,DoubleTree by Hilton,38.920667,-77.227136,Hotel
9,5a1375b246e1b62527257ae3,&pizza,38.917024,-77.223835,Pizza Place


### Assign Venues to Grids

Each location (Tysons Corner, Arlington, & Alexandria) will be segmented into grids of approximately 500 by 500 meters. This represents about a 5 minute walk from one point within the grid to any other point within the grid. As venues are collected from the Foursquare data set, they will be assigned a particular grid ID. Below is a map that shows the Tysons Corner area segmented into a 7x7 grid. 

Then, the `one-hot encoding` technique will be applied and each grid will have a unit score for all categories. The survey data will be applied to the new data frame and the top grid locations will be explored as potential locations for ACME, Inc. This technique is similar to a movie recommendation engine. In this case, the grid locations are equivalent to the movies and the venue categories are equivalent to the movie genres.

In [182]:
lat, lng, rad = PLACES[['lat', 'lng', 'radius-m']].iloc[1,:]

grids = generateGrids(lat, lng, rad, 500, 'tysons')

map = folium.Map(location=[lat,lng], zoom_control=False, zoom_start=13)
folium.map.Marker([lat, lng], popup='<i>Tysons</i>').add_to(map)
for _, row in grids.iterrows():
    folium.vector_layers.Rectangle([row.loc['boundN'], row.loc['boundE']], fill=True, popup=f"<i>{row.loc['grid']}</i>").add_to(map)
map

<a id="methodology"></a>

## Methodology

Each of the new locations for the ACME, Inc. relocation have been defined with an approximate radius of the area that represents the limit from the "city center" the customer is willing to accept. We can use these areas to define our Foursquare queries to limit our search by. So, each of the areas will be searched for venues, and any venues without a category will be discarded. The categories will need to be cleaned and normalized (to appropriately match the employee survey data). 

The grids for each area will be generated and each venue would then be assigned a grid ID. The resulting dataFrame will be a combination of every venue with a grid ID and category. The `one-hot encoding` technique will be used on the categories; and `grid vectors` will be computed by grouping on the grid ID and taking a mean of the category counts to produce weighted grids by category vector. Then we can apply a dot product of the `grid vectors` with the employee survey and find the grids with the highest score(s).


### Step 1: Get Venues for Each Region


In [183]:
# Read in cached data if we have it
venues = pd.DataFrame(columns=['id', 'area', 'venue', 'category', 'lat', 'lng'])
try:
    venues = pd.read_csv(PROJECT.get_file('venues.csv'))
    print('Reading cached data')
except RuntimeError:
    for idx, row in PLACES.iloc[1:, :].iterrows():
        venue_list = getVenues(row.loc['lat'], row.loc['lng'], row.loc['radius-m'], time='any', day='any') + getVenues(row.loc['lat'], row.loc['lng'], row.loc['radius-m'], time='any', day='any', categoryId=EXTRA_CATS)
        df = venues2DataFrame(venue_list)
        df['area'] = row.loc['id']
        venues = venues.append(df, sort=True)
    
    PROJECT.save_data('venues.csv', venues.to_csv(index=False))
    

venues.drop_duplicates(subset = 'id', keep = False, inplace = True)
display(HTML(f'<h4>Sampling of Venues (10 out of {venues.shape[0]})</h4>'))
venues.sample(n=10, random_state=3)

Reading cached data


,area,category,id,lat,lng,venue
267,arlington,Wine Shop,4b2c0063f964a52055bf24e3,38.898468,-77.118400,Arrowine & Cheese
656,alexandria,Café,4adf0a0cf964a520387721e3,38.806516,-77.057119,Uptowner Cafe
102,tysons,Asian Restaurant,584600b307ac071f8eb0ce5f,38.924750,-77.217180,TenPenh Tysons
515,alexandria,Art Gallery,4ad0dc87f964a52087da20e3,38.804865,-77.039905,Torpedo Factory Art Center
410,arlington,Convenience Store,4bd8d87d11dcc9287f1bf833,38.855822,-77.111347,7-Eleven
540,alexandria,Supermarket,5446fbf9498e175d6fb11b4d,38.812730,-77.044038,Harris Teeter
156,arlington,Coffee Shop,5c54463fe55d8b002c112236,38.880331,-77.110234,Philz Coffee
701,alexandria,Post Office,5b3a40cbe55d8b002cd595bf,38.792251,-77.051065,Thornton South Mail Room
353,arlington,Hotel,4ad61eccf964a5202e0521e3,38.894908,-77.072859,Hyatt Centric Arlington
324,arlington,Coffee Shop,4a95242af964a5209c2220e3,38.896574,-77.096578,Starbucks


### Step 2: Clean the Categories

Based on the employee survey we need to combine some of the Foursquare categories into a single category. For instance, there are several different types of restaurants, e.g. `American Restaurant` (see below), that can be combined into just `Restaurant`. We will add a new column to the data frame, `ccategory` to represent the cleaned category. We can go through the categories one-by-one since it is a short list:

*NOTE:* In a few cases (e.g. `Post Office`, or `Metro Station`) we do not actually have any work to do, we can just copy the actual `category` value to the `ccategory`. However, I am simply double checking that these venues actually exist. Which is how the `EXTRA_CATS` variable came about since initial inspection did not find some of the venue categories. You can find more about the Foursquare categories from [here](https://developer.foursquare.com/docs/build-with-foursquare/categories-changelog/)

In [184]:
SURVEY

,Type,Rating
0,Metro Station,9.0
1,Gym,6.5
2,Coffee Shop,7.0
3,Post Office,2.0
4,Cleaners,3.5
5,Sandwich Place,7.0
6,Convenience Store/Drug Store,2.5
7,Bar,8.5


#### Find `Metro Station`

In [185]:
mask = venues['category'] == 'Metro Station'
venues.loc[mask, 'ccategory'] = 'Metro Station'
venues.loc[mask]

,area,category,id,lat,lng,venue,ccategory
125,tysons,Metro Station,50536a58e4b07ba3e5bcb6fd,38.921266,-77.234227,Greensboro Metro Station,Metro Station
137,tysons,Metro Station,50537ac9e4b0d505c238c081,38.928851,-77.241622,Spring Hill Metro Station,Metro Station
392,arlington,Metro Station,44cbc5fdf964a5200d361fe3,38.882963,-77.103344,Virginia Square-GMU Metro Station,Metro Station
393,arlington,Metro Station,49e4c173f964a52037631fe3,38.881960,-77.111551,Ballston-MU Metro Station,Metro Station
413,arlington,Metro Station,4474ca43f964a520b6331fe3,38.887179,-77.095179,Clarendon Metro Station,Metro Station
431,arlington,Metro Station,4a920a71f964a520ad1c20e3,38.891269,-77.085202,Court House Metro Station,Metro Station
436,arlington,Metro Station,4bd6d75e637ba593d2baf870,38.888966,-77.133733,WMATA Orange Line Metro,Metro Station
694,alexandria,Metro Station,4afdf87bf964a520902c22e3,38.814038,-77.053641,Braddock Road Metro Station,Metro Station
697,alexandria,Metro Station,4ace7d6bf964a520d8d020e3,38.806398,-77.060765,King Street - Old Town Metro Station,Metro Station


#### Find `Gym`

In [186]:
mask = venues['category'].str.contains('Gym')
venues.loc[mask, 'ccategory'] = 'Gym'
venues.loc[mask].head()

,area,category,id,lat,lng,venue,ccategory
11,tysons,Gym / Fitness Center,4b30a8abf964a520b6fa24e3,38.923286,-77.230169,Tysons Sport&Health,Gym
34,tysons,Gym,4b92725ef964a52032fa33e3,38.912601,-77.224865,Equinox Tysons Corner,Gym
44,tysons,Gym / Fitness Center,5228caac11d2ddac1097f1ee,38.921538,-77.236464,Crunch Fitness - Tysons Corner,Gym
88,tysons,Gym / Fitness Center,4aff16e5f964a520063422e3,38.918279,-77.215657,McLean Sport&Health,Gym
90,tysons,Gym,4d3afad897e0a1cd1b6f733e,38.929979,-77.236325,UFC Gym,Gym


#### Find `Coffee Shop`

In [187]:
mask = venues['category'].str.contains('Coffee')
venues.loc[mask, 'ccategory'] = 'Coffee Shop'
venues.loc[mask].head()

,area,category,id,lat,lng,venue,ccategory
7,tysons,Coffee Shop,5dd831fed892900007dfa839,38.917435,-77.223452,Shotted Specialty Coffee,Coffee Shop
33,tysons,Coffee Shop,4b566252f964a520220e28e3,38.921646,-77.236275,Starbucks,Coffee Shop
46,tysons,Coffee Shop,4a82c355f964a52051f91fe3,38.918554,-77.222471,Starbucks,Coffee Shop
63,tysons,Coffee Shop,4ff18f9ee4b0279cf3414a32,38.924531,-77.223434,Entyse Espresso Bar,Coffee Shop
65,tysons,Coffee Shop,4bf1af1f99d02d7f7c6cc948,38.918380,-77.220390,Starbucks,Coffee Shop


#### Find `Post Office`

In [188]:
mask = venues['category'] == 'Post Office'
venues.loc[mask, 'ccategory'] = 'Post Office'
venues.loc[mask]

,area,category,id,lat,lng,venue,ccategory
126,tysons,Post Office,5c40f14f4aa3f8002c9ffa92,38.916130,-77.226370,Tysons Self Serve Po,Post Office
129,tysons,Post Office,4b3fae94f964a52014ac25e3,38.929061,-77.237483,US Post Office,Post Office
139,tysons,Post Office,5a0deb0bfebf312ba8db0007,38.932800,-77.228944,Freddie Mac Mail Room,Post Office
386,arlington,Post Office,4b44bb2cf964a52091fa25e3,38.885503,-77.095286,US Post Office,Post Office
400,arlington,Post Office,4bb37cd814cfd13a851716ab,38.873605,-77.104322,US Post Office,Post Office
405,arlington,Post Office,505c9e24e4b0084c3ffaf677,38.883914,-77.083172,Fort Myer Post Office,Post Office
409,arlington,Post Office,4b50ae7df964a5203c2d27e3,38.891978,-77.083704,US Post Office,Post Office
418,arlington,Post Office,4c3388d8213c2d7fb11c375d,38.885639,-77.141428,US Post Office,Post Office
427,arlington,Post Office,5277d24b498e60adefd6ecfd,38.883208,-77.082773,Ft Myer Post Office,Post Office
429,arlington,Post Office,4b55ca1ef964a52017f027e3,38.894871,-77.132883,US Post Office,Post Office


#### Find `Cleaners`

In [189]:
mask = venues['category'].str.contains('Cleaner')
venues.loc[mask, 'ccategory'] = 'Cleaners'
venues.loc[mask].head()

,area,category,id,lat,lng,venue,ccategory
138,tysons,Dry Cleaner,5927cf0f12f0a919d91a674c,38.915293,-77.214388,Metro Carpet & Upholstery,Cleaners
412,arlington,Dry Cleaner,53b589b2498e8be468ec65a0,38.885839,-77.094895,A1 Clarendon Valet & Shoe Repair,Cleaners
417,arlington,Dry Cleaner,5507087f498e67b7910434b2,38.877199,-77.124149,Clean All Laundromat and Dry Cleaners,Cleaners
424,arlington,Dry Cleaner,58ac8ff5102f472e5f85c5e9,38.880766,-77.086053,Pershing Cleaners,Cleaners
698,alexandria,Dry Cleaner,5348576011d21df2b73b1517,38.813283,-77.056105,Yates Dry Cleaning,Cleaners


#### Find `Sandwich Place`

In [190]:
mask = venues['category'] == 'Sandwich Place'
venues.loc[mask, 'ccategory'] = 'Sandwich Place'
venues.loc[mask].head()

,area,category,id,lat,lng,venue,ccategory
81,tysons,Sandwich Place,51001e54e4b0240c497967ea,38.927578,-77.241990,Potbelly Sandwich Shop,Sandwich Place
86,tysons,Sandwich Place,57867967498e2aaa98f18b84,38.931832,-77.238403,Sir sandwich co.,Sandwich Place
97,tysons,Sandwich Place,5c1691bea4b51b002cb9da8a,38.930065,-77.241771,Jersey Mike's Subs,Sandwich Place
220,arlington,Sandwich Place,4a9fe69bf964a520833d20e3,38.881585,-77.112388,Potbelly Sandwich Shop,Sandwich Place
234,arlington,Sandwich Place,4aa2ce88f964a520434220e3,38.890098,-77.089906,Earl's Sandwiches,Sandwich Place


#### Find `Convenience Store/Drug Store`

In [191]:
mask = (venues['category'] == 'Convenience Store') | (venues['category'] == 'Miscellaneous Shop') # turns out that the drug store category is not labeled right and comes back as Miscellaneous Shop
venues.loc[mask, 'ccategory'] = 'Convenience Store/Drug Store'
venues.loc[mask].head()

,area,category,id,lat,lng,venue,ccategory
127,tysons,Convenience Store,4c02ac01187ec928688bb47b,38.911672,-77.221579,7-Eleven,Convenience Store/Drug Store
130,tysons,Miscellaneous Shop,5a302517f8cbd4709f6d835a,38.922357,-77.233817,Boro Lobby Shop,Convenience Store/Drug Store
131,tysons,Convenience Store,4bb7bc8053649c746c3146fb,38.922685,-77.237278,7-Eleven,Convenience Store/Drug Store
134,tysons,Convenience Store,542b1aa1498eabd819b5121a,38.918122,-77.221432,The Market at Tysons Corner,Convenience Store/Drug Store
136,tysons,Convenience Store,548b12ce498e2e25dfca4fc1,38.915377,-77.218019,Tycon Lobby Shop,Convenience Store/Drug Store


#### Find `Bar`

In [192]:
mask = venues['category'] == 'Bar'
venues.loc[mask, 'ccategory'] = 'Bar'
venues.loc[mask].head()

,area,category,id,lat,lng,venue,ccategory
128,tysons,Bar,5c5b379ce65d0c002c0563e9,38.920788,-77.225424,Tysons Social Tavern,Bar
132,tysons,Bar,5dbc9b1b82b46d0008d315f7,38.924898,-77.233546,High Point,Bar
140,tysons,Bar,4a85ec98f964a520acff1fe3,38.917502,-77.219752,TGI Fridays,Bar
147,tysons,Bar,4bad6efaf964a52014503be3,38.917789,-77.219770,La Sandia - Tysons Corner,Bar
378,arlington,Bar,5bf20a1f1cf2e1002c86c279,38.879729,-77.109933,Punch Bowl Social,Bar


#### FInal Cleanup

Examine categories with more than 1 instance and see if we need to clean any of those, e.g.:
* Pharmacy -> Drug Store
* Burger Joint -> Sandwich Place
* Yoga Studio -> Gym


Anything else that does not fit into those categories we can drop.

In [193]:
venues.loc[venues['ccategory'].isnull(), 'category'].value_counts()

Park                          21
Hotel                         19
American Restaurant           19
Pizza Place                   16
Grocery Store                 13
Gas Station                   11
Bakery                        11
Middle Eastern Restaurant     10
Pharmacy                      10
Mexican Restaurant            10
Burger Joint                   9
Italian Restaurant             8
Spa                            7
Seafood Restaurant             7
Café                           7
Food Truck                     6
New American Restaurant        6
Mediterranean Restaurant       6
Ice Cream Shop                 6
Trail                          5
Yoga Studio                    5
Chinese Restaurant             5
Sushi Restaurant               5
Donut Shop                     5
Wine Bar                       5
Department Store               4
Japanese Restaurant            4
Sports Bar                     4
Salad Place                    4
Cocktail Bar                   4
          

In [194]:
venues.loc[venues['category'] == 'Pharmacy', 'ccategory'] = 'Convenience Store/Drug Store'
venues.loc[(venues['category'] == 'Deli / Bodega') | (venues['category'] == 'Burger Joint') | (venues['category'] == 'Diner'), 'ccategory'] == 'Sandwich Place'
venues.loc[venues['category'] == 'Yoga Studio', 'ccategory'] == 'Gym'
venues.loc[(venues['category'] == 'Café') | (venues['category'] == 'Donut Shop') | (venues['category'] == 'Bagel Shop'), 'ccategory'] = 'Coffee Shop'
venues.loc[venues['category'].str.contains('Bar'), 'ccategory'] = 'Bar'

final_venues = venues[~venues['ccategory'].isnull()].reset_index()

### Step 3: Assign Venues to Grids

We start by creating a series for grids for each area, then we will test each venue's lat/lng to determine which grid it belongs to. *NOTE:* as an afterthought, this would have been easier by using geohashing.

If we examine the venue points and grid locations we will see veunes that lie outside our target areas. For this case we will simply exclude them.

In [195]:
grids = pd.DataFrame(columns=['grid', 'boundN', 'boundE'])
for _, place in PLACES.iloc[1:, :].iterrows():
    grids = grids.append(generateGrids(place.loc['lat'], place.loc['lng'], place.loc['radius-m'], 500, place.loc['id']))

grids.head()

,grid,boundN,boundE
0,tysons_0,"(38.92926587771606, -77.24464714602904)","(38.92608976404745, -77.24056460410816)"
1,tysons_1,"(38.92926587771606, -77.24056460410816)","(38.92608976404745, -77.23648206218729)"
2,tysons_2,"(38.92926587771606, -77.23648206218729)","(38.92608976404745, -77.23239952026643)"
3,tysons_3,"(38.92926587771606, -77.23239952026643)","(38.92608976404745, -77.22831697834556)"
4,tysons_4,"(38.92926587771606, -77.22831697834556)","(38.92608976404745, -77.2242344364247)"


In [196]:
map = folium.Map(location=CENTER, zoom_control=False, zoom_start=11)

for _, row in grids.iterrows():
    folium.vector_layers.Rectangle([row.loc['boundN'], row.loc['boundE']], fill=True, popup=f"<i>{row.loc['grid']}</i>").add_to(map)
    
for _, row in final_venues.iterrows():
    folium.map.Marker([row.loc['lat'], row.loc['lng']], popup=f'<i>{row.loc["venue"]}</i>').add_to(map)
map

In [197]:
grid_ids = []
for _, venue in final_venues[['lat','lng']].iterrows():
    found = False
    for _, grid in grids.iterrows():
        if checkBounds(grid.loc['boundN'], grid.loc['boundE'], venue.loc['lat'], venue.loc['lng']):
            grid_ids.append(grid.loc['grid'])
            found = True
            break
    if not found:
        grid_ids.append('none')

grid_ids[:10]            

['tysons_26',
 'tysons_10',
 'tysons_32',
 'tysons_16',
 'tysons_39',
 'tysons_16',
 'tysons_26',
 'tysons_32',
 'tysons_12',
 'tysons_26']

In [198]:
final_venues['grid'] = grid_ids
venues_of_interest = final_venues[final_venues['grid'] != 'none']

In [199]:
display(venues_of_interest.sample(n=10, random_state=1))
print(f'Total number of venues: {venues_of_interest.shape[0]}')

,index,area,category,id,lat,lng,venue,ccategory,grid
55,229,arlington,Coffee Shop,4ab5103ef964a520437220e3,38.883034,-77.103661,Starbucks,Coffee Shop,arlington_91
58,251,arlington,Gym / Fitness Center,4f3bb0d1e4b036beb8d9f305,38.883554,-77.117112,Ballston Crossfit,Gym,arlington_74
136,470,arlington,Sports Bar,59e95631065ef57a21e9c1a8,38.887420,-77.094840,The G.O.A.T,Bar,arlington_65
93,405,arlington,Post Office,505c9e24e4b0084c3ffaf677,38.883914,-77.083172,Fort Myer Post Office,Post Office,arlington_82
200,729,alexandria,Bar,41366280f964a520c51a1fe3,38.805996,-77.054512,Rock It Grill,Bar,alexandria_26
11,79,tysons,Coffee Shop,57bb4635498e3381b0eb7246,38.924801,-77.217411,Bourbon Coffee,Coffee Shop,tysons_13
144,485,arlington,Hookah Bar,52ffbb63498e67ef26d420f1,38.895386,-77.091006,Cloud Lounge,Bar,arlington_38
46,174,arlington,Café,5cb07a44b9a5a8002c3a4b7d,38.878964,-77.112446,Good Company Doughnuts & Cafe,Coffee Shop,arlington_103
183,689,alexandria,Post Office,4b9e4d0cf964a52054d836e3,38.801982,-77.061282,US Post Office,Post Office,alexandria_32
23,125,tysons,Metro Station,50536a58e4b07ba3e5bcb6fd,38.921266,-77.234227,Greensboro Metro Station,Metro Station,tysons_16


Total number of venues: 161


### Step 4: Generate Grid Scores

We start by using the `one-hot encoding` method to generate a data frame of `grid` and `ccategory` counts. Then we can compute the `sum` of each category, grouped by the `grid`. In this case we use `sum` so that it will drive up the score for that given `grid`, i.e. the more `Coffee Shop` options in a particular grid the more attractive that grid becomes. Finally we will normalize to get all scores within a 0-1 range.

In [200]:
grid_counts = pd.get_dummies(venues_of_interest[['grid', 'ccategory']], columns=['ccategory'], prefix='', prefix_sep='')
grid_counts = grid_counts.groupby(['grid']).sum().reset_index()

mmScalar = preprocessing.MinMaxScaler()
norm = mmScalar.fit_transform(grid_counts.drop(['grid'], axis=1).astype('float64').values)
norm

grid_scores = pd.DataFrame(columns = grid_counts.columns)
grid_scores['grid'] = grid_counts['grid']
grid_scores[grid_scores.columns[1:]] = norm
grid_scores.set_index('grid', inplace=True)

display(HTML('<h4>Sample of 10 Grids and Their Scores'))
grid_scores.sample(n=10, random_state=1)

,Bar,Cleaners,Coffee Shop,Convenience Store/Drug Store,Gym,Metro Station,Post Office,Sandwich Place
grid,,,,,,,,
arlington_104,0.2,0,0.333333,0,0,0,0,0
arlington_96,0,1,0,0,0,0,0,0
alexandria_37,0.6,0,0.666667,0,0,0,0,0
tysons_40,0,0,0,0.5,0,0,0,0
arlington_92,0,0,0,0.5,0,0,0,0
arlington_155,0,0,0,0.5,0,0,0,0
arlington_54,0.2,0,0.333333,1,0,1,0.5,0.333333
arlington_23,0,0,0.333333,0.5,0,0,0,0
arlington_27,0,0,0,0,1,0,0,0


Now apply the `SURVEY` results to create the grid rating and apply the rating back to the venues of interest

In [202]:
survey = SURVEY.set_index('Type').T.reset_index(drop=True).iloc[0]
ratings = ((grid_scores*survey).sum(axis=1))/(survey.sum())
ratings = pd.DataFrame(ratings).rename(columns={0: 'rating'})

recos = grid_counts.join(ratings, on='grid').set_index('grid')
recos.sort_values(by='rating', ascending=False).head()

,Bar,Cleaners,Coffee Shop,Convenience Store/Drug Store,Gym,Metro Station,Post Office,Sandwich Place,rating
grid,,,,,,,,,
arlington_65,2,0,1,1,1,1,0,0,0.488768
alexandria_32,1,0,1,2,1,0,2,3,0.478986
tysons_16,1,0,1,1,1,1,0,0,0.451812
alexandria_24,1,0,1,0,1,1,0,0,0.424638
arlington_54,1,0,1,2,0,1,1,1,0.410145


<a id="results"></a>

## Results

Now we have a list of grids and their rating based on the survey data. We can now look at the top `N` grids as recommendations to ACME, Inc. on their new location. We can show them geographically where the grids will be and what venues are in those locations. And since `Metro Station` was their top priority, we will also indicate which top grids have access to a `Metro Station` or not.


In [229]:
geo_recos = grids.join(recos, on='grid').sort_values('rating', ascending=False).head(10)
venues_recos = venues_of_interest[venues_of_interest['grid'].isin(geo_recos['grid'])]

map = folium.Map(location=CENTER, zoom_control=False, zoom_start=11)

for idx, row in geo_recos.iterrows():
    color = 'blue' if idx == 0 else 'green'
    folium.vector_layers.Rectangle([row.loc['boundN'], row.loc['boundE']], color=color if row.loc['Metro Station'] else 'red', fill=True, popup=f"<i>{row.loc['grid']}</i>").add_to(map)
    
for _, row in venues_recos.iterrows():
    icon = folium.Icon(color='green') if row.loc['ccategory'] == 'Metro Station' else None
    folium.map.Marker([row.loc['lat'], row.loc['lng']], icon=icon, popup=f'<i>{row.loc["venue"]}</i>').add_to(map)
    
display(HTML('''
<h4 style="text-align: center">Top 10 Recommended Locations</h4>
<i style="color: blue">Top Scoring Location</i>
<br>
<i style="color: green">Grids Containing Metro Stations</i>
<br>
<i style="color: red">Grids without Metro Stations</i>
<br>
Green Markers indicate locations of Metro Stations
'''))
display(map)

<a id="discussion"></a>

## Discussion

One might notice that the top rated location only has two venues, this is due to a few factors:
* The desire to have a `Metro Station` is very high compared to all other venues
* The number of venues per grid is not currently taken into account as a feature. We could include the number of venues as part of the calculation and ask ACME, Inc. to determine how important it is to have more variety. That way a grid with only two venues of interest would not be the top choice.

Also, there could be other data to help with the analysis as well. For instance, the `Metro Station` in Tysons Corner are new (within the last 3 years); new venues will most likely emerge as more and more businesses move there. However, this might also drive up property and rent values; another source of data that could provide greater insight. These factors might further inform ACME, Inc.'s final decision.

<a id="conclusion"></a>

## Conclusion

This method provided an easy and extensible way to provide location recommendations based on survey data. This could be further extended for other categories as well as applying some of the findings of the [Discussion](#discussion) section. Also, much of the data cleansing could be easily automated. 